In [1]:
import numpy as np 
import pandas as pd 
import re
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
df = pd.read_excel('/kaggle/input/song-th-en-200/Top_song_200_4_27_24 (1).xlsx')   ###
df.head()

,Rank,Name Song,Artist,Peak Rank,Previous Rank,Weeks On Chart,Streaming,Uri,Lyrics,Lyrics Cleaned Ver1,Lyrics Translated,Unnamed: 11
0,1,Like Crazy,Jimin,1,2,56,2116360,https://open.spotify.com/track/3Ua0m0YmEjrMi9X...,[Intro]\n(I think we could last forever)(I'm a...,"She's saying, Baby, come and follow me \nTher...",“เธอพูดว่า ที่รัก มาตามฉันมาสิ”\nคืนนี้ไม่มีอะ...,NaN
1,2,Standing Next to You,Jung Kook,1,3,7,2036630,https://open.spotify.com/track/2KslE17cAJNHTsI...,Standing next to you\nPlay me slow\nI push up ...,Standing next to you\nPlay me slow\nI push up ...,ยืนอยู่ข้าง ๆ คุณ\nเล่นฉันช้า ๆ\nฉันผลักดันควา...,NaN
2,3,FRI(END)S,V,1,1,5,1862164,https://open.spotify.com/track/0PBTlQ84CBJF0tB...,(Friends)\nOoh\n \nYou're in my head\nI had pl...,(Friends)\n\nYou're in my head\nI had plans fo...,(เพื่อน)\n\nคุณอยู่ในหัวของฉัน\nฉันมีแผนสำหรับ...,NaN
3,4,Love Me Again,V,1,5,36,1632169,https://open.spotify.com/track/2N0SPREDYqILVEF...,[Verse 1]\nYou left me\nTook those memories wi...,\nYou left me\nTook those memories with you\nB...,คุณทิ้งฉัน\nได้นำความทรงจำเหล่านั้นติดตัวไปด้ว...,NaN
4,5,Nightwalker,TEN,2,4,10,1280779,https://open.spotify.com/track/4RiudH8RehvLLrk...,"Oh, nearing the hour\nSo I been waiting so lon...",nearing the hour\nSo I been waiting so long\nH...,ใกล้ชั่วโมง\nฉันจึงรอมานานมาก\nซ่อนพลังของฉัน\...,NaN


## preprocessing

In [3]:
def lang_detect(txt):
    lang = set()
    for char in str(txt):
        if '\u0e00' <= char <= '\u0e7f':  # Thai
            lang.add('th')
        elif char.isalpha():  # Default to English
            lang.add('en')
    if len(lang) == 2:
        return  'th en'
    else:
        return list(lang)[0]

In [4]:
df['lang'] = df['Lyrics Cleaned Ver1'].apply(lang_detect)
df['lang'].value_counts()

lang
th       104
en        63
th en     33
Name: count, dtype: int64

In [5]:
thai_df = df[df['lang'] == 'th'][['Name Song','Lyrics Cleaned Ver1']]
thai_df = thai_df.reset_index().iloc[:,1:]
thai_df.head()

,Name Song,Lyrics Cleaned Ver1
0,เผื่อวันไหนเธอจะกลับมา,เธอจากไปตั้งนาน เธอมีใครเข้ามา\nเราเดินแยกทาง ...
1,เพลงที่เธอไม่ฟัง,รู้สึกยังไงเก็บเอาไว้ไม่เคยคิดจะบอก\nเอ่ยปากตร...
2,จำนน (White Flag),ไม่รู้ว่านานเท่าไหร่ก็จำไม่ได้นัก\nที่ฉันจำเป็...
3,เมร่อน,เพื่อนฉันล่ะคนนี้ มันชื่ออีเมอะหอยใหญ่\nน่ารัก...
4,ดาวหางฮัลเลย์,ระหว่างที่มีดาวดวงนึงโคจรในอวกาศ\nจากวันนั้นฉั...


In [6]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 62.4 MB/s eta 0:00:00


In [7]:
import re
import string
import matplotlib as mpl
import matplotlib.pyplot as plt

from pythainlp.corpus import (thai_stopwords, 
                              thai_words,
                              thai_syllables,
                             thai_negations,
                             thai_orst_words)
from pythainlp.corpus.ttc import word_freqs
from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize
from collections import Counter
from tqdm import tqdm
from pythainlp.tokenize import word_tokenize as thai_word_tokenize

In [8]:
def clean_thai(text):
    text = re.sub(r'[\n\rๆ]',' ',text)
    text = re.sub(r'\s+',' ',text)
    translator = str.maketrans("", "", string.punctuation)

    # Apply the translation table to the input string
    result = text.translate(translator)
    return result

words = set(thai_words())
freqswords = word_freqs()
freq = 1000 #Set frequency
freqsResults = []

for line in list(freqswords):
    if line[1] >= freq: #More than threshold means frequency
        freqsResults.append(line[0])

def prepare_thai(text):
    # Tokenization
    tokens = thai_word_tokenize(text, engine='newmm')

    # Remove stopwords
    stop_words = set()
    tokens = [token for token in tokens if token not in stop_words and len(token)>1]
    tokens = [token for token in tokens if token in words]

    return ' '.join(tokens)

def seperate_txt(text):
    max_length = 1000
    split_text = []
    current_chunk = ""

    for word in text.split():
        if len(current_chunk) + len(word) < max_length:
            if current_chunk:
                current_chunk += " " + word
            else:
                current_chunk = word
        else:
            split_text.append(current_chunk)
            current_chunk = word

    # Append the last chunk
    if current_chunk:
        split_text.append(current_chunk)
        
    return split_text

In [9]:
thai_df['token'] = thai_df['Lyrics Cleaned Ver1'].apply(clean_thai)
thai_df['token'] = thai_df['token'].apply(prepare_thai)
thai_df['sep_token'] = thai_df['token'].apply(seperate_txt)

In [10]:
import requests
url = "https://api.aiforthai.in.th/tpos"
 
# params = {'text':'สวัสดีคุณ'}
headers = {
    'Apikey': "uBqPAel8u1k3wE7qeMVHnz1Wpngj6VDR",
    }
 
# response = requests.get(url, headers=headers, params=params)

# data = list(zip(response.json()['words'],response.json()['tags']))[0]
# print(data)
def meaning_clean(txt):
    tokens = []
    for i in txt:
        response = requests.get(url, headers=headers, params={'text':i})
        tokens.extend(list(zip(response.json()['words'],response.json()['tags'])))
    return tokens

In [11]:
words_meaning = []
for i in tqdm(thai_df['sep_token']):
    words_meaning.append(meaning_clean(i))

100%|██████████| 104/104 [04:13<00:00,  2.43s/it]


In [12]:
w_t_total = []
word_mean= []
choosen = ['ADV','AUX','CD','CL','CNJ','COMP','DDEM','DINT','DPER','FXAJ','FXAV','FXG','FXN','FWA','FWN','FWV','FWX','IJ','JJA','JJV','NEG','OD','P','PAR','PDEM','PDT','PINT','PPER','PU','REFX','X','NR']
for j in tqdm(words_meaning):
    w = []
    t = []
    for i in j:
        w.append(i[0])
        t.append(i[1])
    merged_string_w = ''.join(w)
    result_w = merged_string_w.split()
    merged_string_t = ''.join(t)
    result_t = merged_string_t.split()
    for i in range(len(result_w)):
        if result_t[i] not in choosen:
            w_t_total.append(result_w[i])
            
    word_mean.append(w_t_total)
    w_t_total = []
        

100%|██████████| 104/104 [00:00<00:00, 2401.96it/s]


In [13]:
thai_df['meaning_tokens'] = word_mean
thai_df['meaning_tokens'].head()

0    [จากไป, ตั้ง, มี, เข้ามา, เดิน, แยกทาง, แล้วก็...
1    [รู้สึก, ยังไง, เก็บ, เอาไว้, คิด, บอก, เอ่ยปา...
2    [รู้, เท่าไหร่, จำ, จำเป็นต้อง, แอบ, รัก, ต่อใ...
3    [ชื่อ, อะ, หอย, น่ารัก, มี, หัวใจ, กิน, ออก, เ...
4    [มี, ดาว, โคจร, อวกาศ, วันนั้น, มองโลก, เหมือน...
Name: meaning_tokens, dtype: object

In [14]:
thai_corpus = []
for i in thai_df['meaning_tokens']:
    thai_corpus.extend(i)
thai_corpus = pd.Series(list(set(thai_corpus)))

In [15]:
thai_corpus.to_csv('/kaggle/working/thai_words.csv')

## similarity

In [16]:
# !pip install -U sentence-transformers

In [17]:
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# def Sim_score(sentences_th,lda_pro_th,embeddings_en,lda_pro_en):
#     embeddings_th = model.encode(sentences_th, convert_to_tensor=True)
#     embeddings_en = model.encode(sentences_en, convert_to_tensor=True)
#     cosine_scores = util.pytorch_cos_sim(embeddings_th, embeddings_en)
#     score = []
#     for i, th_sentence in enumerate(sentences_th):
#         for j, en_sentence in enumerate(sentences_en):
#             similarity_score = float(cosine_scores[i][j])
#             sim_score = float(lda_pro_th[i] * lda_pro_en[j] * similarity_score)
#             score.append([th_sentence,en_sentence,lda_pro_th[i],lda_pro_en[j],round(similarity_score,3),round(sim_score,3)])
#     return score
    